<a href="https://colab.research.google.com/github/TarkanAl-Kazily/awbw_replay_parser/blob/main/AWBW_Replays_Interactive_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to my AWBW Replay Parser project!

This interactive notebook will introduce you to the replay parsing library, and demonstrate how to use it to learn more about your matches.

This notebook and the remaining source code is all hosted on Github here: [https://github.com/TarkanAl-Kazily/awbw_replay_parser](https://github.com/TarkanAl-Kazily/awbw_replay_parser)

## Prerequisites

Ideally to use this notebook, you have some familiarity with Python and matplotlib if you wish to customize the displayed data you are analyzing. If you wish to contribute and make changes to this notebook or the repo itself, you will need a Github account and make a pull request (following the [contribution guide](https://github.com/TarkanAl-Kazily/awbw_replay_parser))

You will also need an AWBW Replay file that conforms to the newer .zip replay format. To access AWBW Replays, you must have an AWBW account and download them before they are deleted from the site. Because an account is required, it's currently not possible to automatically download replays through this notebook.

In [7]:
#!git clone https://github.com/TarkanAl-Kazily/awbw_replay_parser.git


--2021-12-29 20:22:20--  https://awbw.amarriner.com/replay_download.php?games_id=526198
Resolving awbw.amarriner.com (awbw.amarriner.com)... 165.227.82.216
Connecting to awbw.amarriner.com (awbw.amarriner.com)|165.227.82.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘526198.zip’

526198.zip              [ <=>                ]       0  --.-KB/s    in 0s      

2021-12-29 20:22:21 (0.00 B/s) - ‘526198.zip’ saved [0]

